In [7]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [3]:
facebook_web_log = pd.read_excel('./Time Between Two Events Data.xlsx')

In [14]:
df = facebook_web_log[(facebook_web_log['action'] == 'page_load')
                      |(facebook_web_log['action'] == 'scroll_down')]
df.sort_values(by = ['user_id','timestamp'], inplace = True)
df['event_id'] = df.index

In [17]:
df = df[['event_id','user_id','timestamp','action']]

In [22]:
df_mini = df[df['user_id'] == 1].copy()

In [20]:
df_mini

,event_id,user_id,timestamp,action
8,8,1,2019-04-25 13:40:00,page_load
9,9,1,2019-04-25 13:40:10,scroll_down
10,10,1,2019-04-25 13:40:15,scroll_down
11,11,1,2019-04-25 13:40:20,scroll_down
12,12,1,2019-04-25 13:40:25,scroll_down
13,13,1,2019-04-25 13:40:30,scroll_down
14,14,1,2019-04-26 11:15:00,page_load
15,15,1,2019-04-26 11:15:10,scroll_down
16,16,1,2019-04-26 11:15:20,scroll_down


In [8]:
last_load = pd.to_datetime(np.nan)

In [23]:
df_mini['prev_load'] = -1

In [24]:
df_mini

,event_id,user_id,timestamp,action,prev_load
8,8,1,2019-04-25 13:40:00,page_load,-1
9,9,1,2019-04-25 13:40:10,scroll_down,-1
10,10,1,2019-04-25 13:40:15,scroll_down,-1
11,11,1,2019-04-25 13:40:20,scroll_down,-1
12,12,1,2019-04-25 13:40:25,scroll_down,-1
13,13,1,2019-04-25 13:40:30,scroll_down,-1
14,14,1,2019-04-26 11:15:00,page_load,-1
15,15,1,2019-04-26 11:15:10,scroll_down,-1
16,16,1,2019-04-26 11:15:20,scroll_down,-1


In [31]:
last_load = pd.to_datetime(np.nan)
def l_load(row):
    if row['action'] == 'page_load':
        global last_load
        last_load = row['event_id']
        row['prev_load'] = row['event_id']
    else:
        row['prev_load'] = last_load
    return row['prev_load']


In [32]:
df_mini['prev_load'] = df_mini.apply(l_load,axis=1)


In [33]:
df_mini.merge()

,event_id,user_id,timestamp,action,prev_load
8,8,1,2019-04-25 13:40:00,page_load,8
9,9,1,2019-04-25 13:40:10,scroll_down,8
10,10,1,2019-04-25 13:40:15,scroll_down,8
11,11,1,2019-04-25 13:40:20,scroll_down,8
12,12,1,2019-04-25 13:40:25,scroll_down,8
13,13,1,2019-04-25 13:40:30,scroll_down,8
14,14,1,2019-04-26 11:15:00,page_load,14
15,15,1,2019-04-26 11:15:10,scroll_down,14
16,16,1,2019-04-26 11:15:20,scroll_down,14


In [ ]:
df_mini.merge(df_mini)

In [44]:
df

,user_id,date,date_1,user_id_1,streak
0,u001,2022-08-04,NaN,NaN,0
17,u001,2022-08-05,2022-08-04,u001,0
8,u001,2022-08-06,2022-08-05,u001,0
9,u001,2022-08-07,2022-08-06,u001,0
14,u001,2022-08-08,2022-08-07,u001,0
15,u001,2022-08-09,2022-08-08,u001,0
1,u002,2022-08-05,2022-08-09,u001,0
12,u002,2022-08-06,2022-08-05,u002,0
13,u002,2022-08-07,2022-08-06,u002,0
10,u002,2022-08-08,2022-08-07,u002,0


In [60]:
df

,user_id,date,date_1,user_id_1,streak,streak_1
0,u001,2022-08-04,NaN,NaN,1.0,NaN
17,u001,2022-08-05,2022-08-04,u001,2.0,1.0
8,u001,2022-08-06,2022-08-05,u001,0.0,0.0
9,u001,2022-08-07,2022-08-06,u001,0.0,0.0
14,u001,2022-08-08,2022-08-07,u001,0.0,0.0
15,u001,2022-08-09,2022-08-08,u001,0.0,0.0
1,u002,2022-08-05,2022-08-09,u001,1.0,0.0
12,u002,2022-08-06,2022-08-05,u002,2.0,1.0
13,u002,2022-08-07,2022-08-06,u002,0.0,0.0
10,u002,2022-08-08,2022-08-07,u002,0.0,0.0


In [88]:
def is_streak(user_id,date,date_1,user_id_1,streak,streak_1):
    if streak == 0:
        if user_id != user_id_1:
            return 1
        elif streak_1 != 0:
            if date == date_1 +  timedelta(days=1):
                return streak_1 + 1
            else:
                return streak_1
    return streak

In [59]:
df['streak'] = df.apply(lambda x: is_streak(x['user_id'],x['date'],x['date_1'],x['user_id_1'],x['streak'],x['streak_1']), axis=1)

In [115]:
while 0 in set(df['streak']):
    df['streak_1'] = df['streak'].shift(1)
    df['streak'] = df.apply(lambda x: is_streak(x['user_id'],x['date'],x['date_1'],x['user_id_1'],x['streak'],x['streak_1']), axis=1)


In [116]:
df

,user_id,date,date_1,user_id_1,streak,streak_1
0,u001,2022-08-04,NaN,NaN,1.0,NaN
17,u001,2022-08-05,2022-08-04,u001,2.0,1.0
8,u001,2022-08-06,2022-08-05,u001,3.0,2.0
9,u001,2022-08-07,2022-08-06,u001,4.0,3.0
14,u001,2022-08-08,2022-08-07,u001,5.0,4.0
15,u001,2022-08-09,2022-08-08,u001,6.0,5.0
1,u002,2022-08-05,2022-08-09,u001,1.0,0.0
12,u002,2022-08-06,2022-08-05,u002,2.0,1.0
13,u002,2022-08-07,2022-08-06,u002,3.0,2.0
10,u002,2022-08-08,2022-08-07,u002,4.0,3.0


In [67]:
df.dtypes

user_id       object
date          object
date_1        object
user_id_1     object
streak       float64
streak_1     float64
dtype: object